In [1]:
import numpy as np
import pickle
from sklearn.decomposition import PCA
import tensorflow as tf

##################load data#####################
with open('dataset/preprocessed_data.pickle', 'rb') as f:
    data = pickle.load(f)
    
train_data = data['train_dataset']
test_data = data['test_dataset']

train_labels = data['train_labels']
test_labels = data['test_labels']

# in case you want to select a portion of the features
# start_index=0
# end_index=87
# train_data=train_data[:,start_index:end_index,:]
# test_data=test_data[:,start_index:end_index,:]


input_width = train_data.shape[1]
input_height = train_data.shape[2]

train_data = train_data.reshape((-1, input_width * input_height)).astype(np.float32)
test_data = test_data.reshape((-1, input_width * input_height)).astype(np.float32)

# print("Shape of training set after vectorization:")
# print(train_data.shape)

# print("Shape of test set after vectorization:")
# print(test_data.shape)

################## PCA #####################

print("Running PCA")

# run pca to reduce vector size to 900
pca = PCA(copy=True, iterated_power='auto', n_components=900, random_state=None,
          svd_solver='auto', tol=0.0, whiten=False)

pca.fit(train_data)

train_data = pca.transform(train_data)

test_data = pca.transform(test_data)

print("Shape of training set after pca:")
print(train_data.shape)

print("Shape of test set after pca:")
print(test_data.shape)

################## SVM using TensorFlow #####################

print("Training SVM")

# convert labels to one-hot encoding
train_labels = tf.one_hot(train_labels, depth=10)
test_labels = tf.one_hot(test_labels, depth=10)

# create a SVM model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(900,)),
    tf.keras.layers.Dense(10, use_bias=False, kernel_regularizer=tf.keras.regularizers.l2(0.01))
])

# compile the model
model.compile(optimizer='adam', loss='hinge', metrics=['accuracy'])

# train the model
model.fit(train_data, train_labels, epochs=100, batch_size=128)

# evaluate the model on training set
train_loss, train_acc = model.evaluate(train_data, train_labels, verbose=2)
print(f"Train Accuracy: {train_acc*100:.2f}%")

# evaluate the model on test set
test_loss, test_acc = model.evaluate(test_data, test_labels, verbose=2)
print(f"Test Accuracy: {test_acc*100:.2f}%")

Running PCA
Shape of training set after pca:
(900, 900)
Shape of test set after pca:
(100, 900)
Training SVM
Epoch 1/100
8/8 [==============================] - 0s 4ms/step - loss: 6.5811 - accuracy: 0.1467
Epoch 2/100
8/8 [==============================] - 0s 2ms/step - loss: 5.9717 - accuracy: 0.1733
Epoch 3/100
8/8 [==============================] - 0s 2ms/step - loss: 5.6199 - accuracy: 0.1878
Epoch 4/100
8/8 [==============================] - 0s 2ms/step - loss: 5.3246 - accuracy: 0.2033
Epoch 5/100
8/8 [==============================] - 0s 2ms/step - loss: 5.0636 - accuracy: 0.2267
Epoch 6/100
8/8 [==============================] - 0s 3ms/step - loss: 4.8269 - accuracy: 0.2422
Epoch 7/100
8/8 [==============================] - 0s 2ms/step - loss: 4.6097 - accuracy: 0.2700
Epoch 8/100
8/8 [==============================] - 0s 2ms/step - loss: 4.4041 - accuracy: 0.2844
Epoch 9/100
8/8 [==============================] - 0s 2ms/step - loss: 4.2129 - accuracy: 0.3078
Epoch 10/100
8/8 [